# Notebook `Data Extractor`
***

In [24]:
from yarl import URL
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from pathlib import Path
from src.modules.scraping_functions import *
from itertools import chain
import polars as pl

**Note** : dans le folder `data`, tu trouveras le fichier `wine_links.csv`, il contient l'ensemble des href trouvés après le lancement du module `scraper.py`

- Dans le fichier actuel, il y en a *540*, mais en modifiant les **range** de la fonction `create_urls_browse_list` du module `scraping_functions.py`, tu peux récupérer l'ensemble des hrefs.
- `⚠` ! Pour récupérer les hrefs il faut obligatoirement faire tourner le module `scraper.py` car les hrefs sont générés post-javascript (évidemment ça ne marche pas dans les notebooks...) mais pour les caractéristiques des vins, prix et autres, c'est avant le javascript que c'est généré donc tu peux le faire ici ! 

In [19]:
root = Path(".").resolve()
data_folder = root / "data"
file_path = data_folder / "wine_links.csv"
session = create_session()

Autrement dit la fonction ci-dessous, `create_all_wine_urls` ne fonctionnera pas ici

In [23]:
# create_all_wine_urls()

Donc on importe plutôt les données du folder data en polars puis on retransforme en liste

In [4]:
all_wine_urls = list(
    chain.from_iterable(pl.read_csv(file_path, has_header=False).to_numpy().tolist())
)

Enfin, on extrait tout le contenu des pages avec `extract_all_pages`. Ensuite..c'est à toi ! 🤪

In [ ]:
all_html_pages = extract_all_pages(session, all_wine_urls)

liste des caractéristiques trouvées :

- Titre
- Photo
- Contenance
- Type (Vin blanc, Vin rouge, etc)
- Région
- Volume alcool (%)
- Prix
- Adjectifs ["Fruité", "Vif", "Vignerons passionnés"]
- Millésime
- Cépage
- Gouts
- est bio
- haute valeur environementale
- vigneron indépendant
- est une nouveauté sur le site (tire peut-être le prix vers le bas)
- notes clients
- nb avis
- et bien d'autres j'imagine.....


In [18]:
all_html_pages[2]

'\n<!DOCTYPE HTML><html lang="fr"><head><meta charset="utf-8" /><title>Acheter PINOT GRIS de SOPHIE SCHAAL sur VINATIS au meilleur prix !</title><meta name="description" content="Acheter PINOT GRIS de SOPHIE SCHAAL au meilleur prix sur VINATIS.COM, leader de la vente en ligne. Livraison possible en 24h !" /><meta name="keywords" content="pinot gris 2022, sophie schaal, vin blanc, alsace, alsace aoc, 13 % vol, 100% pinot gris" /><meta name="robots" content="index,follow" /><meta name="viewport" content="width=device-width, minimum-scale=0.25, maximum-scale=1.6, initial-scale=1.0" /><meta name="apple-mobile-web-app-capable" content="yes" /><meta property="og:title" content="PINOT GRIS 2022 - SOPHIE SCHAAL"><meta property="og:image" content="https://www.vinatis.com/84343-large_default/pinot-gris-2022-sophie-schaal.png"><meta property="og:site_name" content="VINATIS"><meta property="og:type" content="product"><meta property="og:description" content="Acheter PINOT GRIS de SOPHIE SCHAAL au m

## 🐱‍💻 Quelques tests, module `soup` à faire

In [9]:
content_page = all_html_pages[535]

# print(content_page)
soup = BeautifulSoup(content_page, "html.parser")
print(soup.find("h1", id="produit-titre").text)
# print(soup.find_all("a", attrs={"class": "maillage"}))
print(soup.find("span", id="our_price_display").text)
print(
    [element.text for element in soup.find_all("span", id="quantity_discount_pretaxe")]
)

print(
    [
        element.text
        for element in soup.find_all(
            "div",
            attrs={
                "class": "table-cell-css vertical-align-top padding-vertical-5 nowrap padding-right-10 taille-xs color-gray-darker"
            },
        )
    ]
)

print(
    [
        element.text
        for element in soup.find_all(
            "div",
            attrs={
                "class": "table-cell-css vertical-align-top padding-vertical-5 taille-xs color-gray-darker text-bold"
            },
        )
    ]
)


print(
    soup.find(
        "div",
        attrs={"class": "margin-top-5 hide-xs"},
    ).text
)


print(
    soup.find_all(
        "a",
        attrs={
            "class": "text-bold padding-5.no-padding-bottom color-green top-label-shadow-green inline-block taille-xs text-uppercase"
        },
    )
)

 MONTEPULCIANO D'ABRUZZO RISERVA 2019 - ANTICO BLASONE
  10,90 €    
[' 8,70 € ']
[' \xa0\xa0Millésime', ' \xa0\xa0Cépage', ' \xa0\xa0Goûts', ' \xa0\xa0Par Goûts', " \xa0\xa0A l'oeil", ' \xa0\xa0Au nez', ' \xa0\xa0En bouche', ' \xa0\xa0Température de service', ' \xa0\xa0Service', ' \xa0\xa0Conservation', ' \xa0\xa0A boire à partir de', " \xa0\xa0Jusqu'à", ' \xa0\xa0Accords mets-vin', ' \xa0\xa0Accords recommandés']
[' 2019', ' 100% Montepulciano', ' Rouge Elégant et racé', ' Fruité', ' Robe couleur rubis aux reflets grenat', ' Arômes de cerise évoluant sur des notes de mûre et de réglisse.', ' Bouche herbacée, vin rond et puissant', ' 18-20°C', ' Ouvrir 1h avant le service', ' A boire et à garder', ' 2019', ' 2025', ' Viande rouge, Gibier, Champignon, Cuisine du monde, Fromage', ' Lasagnes au prosciutto et à la mozzarella fumée, gibiers, grillades']
 Découverte Gourmandise Velouté
[]
